In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

In [ ]:
%%yaml inference_config

log:
    root: '?'
    log_interval: 50
    track_image_level: True
    track_pixel_level: True

model:
    exp_root: /storage/vbutoi/scratch/ESE/training/09_19_23_OxPets_MASSIVEBatchSz
    num_workers: 1  

dataset:
    split: cal 
    input_type: image 

calibration:
    bin_weightings:
        - proportional
        - uniform
    conf_interval_start: 0.0
    conf_interval_end: 1.0
    num_bins: 10

In [ ]:
%%yaml cal_metrics_cfg 

cal_metrics:
    - ECE:
        func: ese.experiment.metrics.ECE
    - TL_ECE:
        func: ese.experiment.metrics.TL_ECE
    - CW_ECE:
        func: ese.experiment.metrics.CW_ECE
    - TENCE:
        func: ese.experiment.metrics.TENCE

In [ ]:
from ionpy.util import dict_product, Config

# Need to define the experiment name
exp_name = "11_10_23_OxPets_UniformityConds"

# Get the options for WMH
##################################################
dataset_options = {
    'log.root': [f'/storage/vbutoi/scratch/ESE/inference/{exp_name}'],
}

base_cfg = Config(inference_config).update(cal_metrics_cfg)

cfgs = []
for cfg_update in dict_product(dataset_options):
    new_cfg = base_cfg.update(cfg_update)
    cfgs.append(new_cfg)

In [ ]:
# from ese.experiment.analysis.inference import get_cal_stats
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# get_cal_stats(cfgs[0])

In [ ]:
from ionpy.slite.submit import submit_jobs
from ese.experiment.analysis.inference import get_cal_stats

submit_jobs(
    project="ESE",
    exp_name=exp_name,
    config_list=cfgs, 
    job_func=get_cal_stats,
    available_gpus=["1"]
    )